### Part 1: Dataset Loading and Preprocessing

#### Objective
The initial step in our project on link prediction in dynamic and complex networks involves loading and preprocessing the dataset. The dataset comprises edges from a fictionalized "Facebook" social network, representing connections between individuals (nodes).

#### Implementation
We begin by importing the necessary `pandas` library, which offers powerful data manipulation capabilities ideal for handling tabular data. Using `pandas`, we load the dataset from a text file where each line represents an edge between two nodes, identified by their numerical IDs. These edges are delimited by whitespace.

To ensure data quality, we remove duplicate edges if any. This preprocessing step is crucial for maintaining the integrity of our network graph, ensuring that it accurately represents the social network without redundant connections.

In [12]:
import pandas as pd

# Load the dataset
edges_path = "facebook_combined.txt"
edges = pd.read_csv(edges_path, delim_whitespace=True, header=None, names=["source", "target"])

# Remove duplicate edges
edges = edges.drop_duplicates()
edges.head()

,source,target
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


### Part 2: Data Splitting for Training and Testing

#### Objective
After loading and preprocessing the dataset, the next critical step involves splitting the data into training and testing sets. This process is pivotal for evaluating the performance of our link prediction algorithms accurately. By training on one subset of the data and testing on a separate subset, we can assess how well our algorithms generalize to unseen data.

#### Implementation
We use the `train_test_split` function from the `sklearn.model_selection` module, a standard practice in machine learning for dividing datasets. We allocate 80% of the data to the training set and 20% to the test set, ensuring a substantial amount of data for both training our models and evaluating their performance.

This split also reflects a real-world scenario where we know some connections (training set) and aim to predict others (test set).


In [13]:
from sklearn.model_selection import train_test_split

# Split the edges into a training set and a test set
train_edges, test_edges = train_test_split(edges, test_size=0.20, random_state=42)

# Output the sizes of the training and test sets to ensure the split was successful
train_edges.shape[0], test_edges.shape[0]

(70587, 17647)

### Part 3: Graph Construction

#### Objective
The cornerstone of our link prediction project is the construction of a graph representing the social network. This graph lays the foundation for computing similarity scores between nodes, which will drive our predictions. By modeling the network as a graph, we leverage the relationships between nodes to infer potential future connections.

#### Implementation
We employ the `networkx` library, a powerful tool for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks. From our training set, we construct an undirected graph where nodes represent individuals and edges depict existing relationships between them.

This graph serves as a map of the network, encapsulating the connectivity information necessary for calculating similarity measures between nodes—a critical step for link prediction.

In [14]:
import networkx as nx

# Construct a graph from the training edges
G = nx.from_pandas_edgelist(train_edges, 'source', 'target')

### Part 4: Calculating Similarity Scores

#### Objective
A pivotal step in link prediction is to calculate similarity scores between pairs of nodes. These scores estimate the likelihood of a link forming between two nodes based on their current connections within the network. We explore various metrics to capture different aspects of similarity, enhancing our predictive capabilities.

#### Implementation
We focus on four similarity metrics:
- **Jaccard Coefficient**: Considers the size of the intersection of the neighbors of two nodes relative to the size of their union.
- **Adamic-Adar Index**: Weighs common neighbors by their connectivity, giving higher importance to shared neighbors with fewer connections.
- **Resource Allocation Index**: Similar to Adamic-Adar but focuses on the shared resources between two nodes.
- **Preferential Attachment**: Assumes that nodes with high degrees are more likely to connect.

Using `networkx`, we calculate these scores for a subset of node pairs. This approach allows us to understand the underlying connectivity patterns and predict future links effectively.


In [15]:
import itertools

# Generate a limited list of node pairs (non-edges) for demonstration
node_pairs = list(itertools.islice(nx.non_edges(G), 500))

# Calculate similarity scores using four metrics
jaccard_coefficients = list(nx.jaccard_coefficient(G, node_pairs))
adamic_adar_index = list(nx.adamic_adar_index(G, node_pairs))
resource_allocation_index = list(nx.resource_allocation_index(G, node_pairs))
preferential_attachment = list(nx.preferential_attachment(G, node_pairs))

# Example output of the first 5 scores for each metric
(jaccard_coefficients[:5], adamic_adar_index[:5], resource_allocation_index[:5], preferential_attachment[:5])

([(0, 35, 0.0),
  (0, 40, 0.08996539792387544),
  (0, 76, 0.007067137809187279),
  (0, 88, 0.04946996466431095),
  (0, 89, 0.0176678445229682)],
 [(0, 35, 0),
  (0, 40, 8.726026520364215),
  (0, 76, 0.9708900350736935),
  (0, 88, 6.16411648968573),
  (0, 89, 3.1899554898721294)],
 [(0, 35, 0),
  (0, 40, 1.4368840349394962),
  (0, 76, 0.2681818181818182),
  (0, 88, 1.4839251529344408),
  (0, 89, 1.0214285714285714)],
 [(0, 35, 283), (0, 40, 9056), (0, 76, 566), (0, 88, 3962), (0, 89, 1415)])

### Part 5: Evaluating Prediction Precision

#### Objective
To assess the effectiveness of our link prediction algorithms, we evaluate their precision. Precision measures the proportion of true positive predictions in a set of top-ranked link predictions. This metric helps us understand how accurately our algorithms can identify future connections within the network.

#### Implementation
We define a function to calculate the precision of our predictions, focusing on the top N recommendations. This function compares the predicted links against the actual links in the test set, providing insight into the predictive power of our similarity metrics.

We then sort the similarity scores from our four metrics in descending order and calculate the precision for each. This process allows us to compare the performance of different similarity measures directly and identify which metrics are most effective for link prediction in our social network graph.

In [16]:
# Define a function to calculate precision
def calculate_precision(recommendations, test_edges_set, top_n=60):
    top_recommendations = recommendations[:top_n]
    true_positives = sum(
        1 for rec in top_recommendations if (rec[0], rec[1]) in test_edges_set or (rec[1], rec[0]) in test_edges_set
    )
    precision = true_positives / top_n
    return precision


# Convert the test edges to a set for faster lookup
test_edges_set = set(map(tuple, test_edges.to_numpy()))

# Sort the similarity scores in descending order
jaccard_sorted = sorted(jaccard_coefficients, key=lambda x: x[2], reverse=True)
adamic_adar_sorted = sorted(adamic_adar_index, key=lambda x: x[2], reverse=True)
resource_allocation_sorted = sorted(resource_allocation_index, key=lambda x: x[2], reverse=True)
preferential_attachment_sorted = sorted(preferential_attachment, key=lambda x: x[2], reverse=True)


# Calculate precision for each algorithm
jaccard_precision = calculate_precision(jaccard_sorted, test_edges_set)
adamic_adar_precision = calculate_precision(adamic_adar_sorted, test_edges_set)
resource_allocation_precision = calculate_precision(resource_allocation_sorted, test_edges_set)
preferential_attachment_precision = calculate_precision(preferential_attachment_sorted, test_edges_set)

(jaccard_precision, adamic_adar_precision, resource_allocation_precision, preferential_attachment_precision)

(0.9666666666666667,
 0.9333333333333333,
 0.9166666666666666,
 0.03333333333333333)

### Part 6: Combining Similarity Scores for Enhanced Predictions

#### Objective
While individual similarity metrics offer valuable insights, combining them can potentially enhance prediction accuracy. This part explores creating a composite score that leverages multiple metrics, aiming to capture a more nuanced understanding of node similarity.

#### Implementation
We calculate a combined score by averaging the Jaccard Coefficient, Adamic-Adar Index, and Resource Allocation Index for each node pair. This approach assumes equal importance of each metric, aiming to balance their individual strengths and weaknesses.

The combined score represents an integrated measure of similarity, potentially improving our ability to predict future links by drawing on the complementary information provided by each metric.

In [17]:
# Create a dictionary to hold the combined scores for each node pair
combined_scores = {}

for pair in node_pairs:
    jaccard_score = next((x[2] for x in jaccard_sorted if x[:2] == pair), 0)
    adamic_adar_score = next((x[2] for x in adamic_adar_sorted if x[:2] == pair), 0)
    resource_allocation_score = next((x[2] for x in resource_allocation_sorted if x[:2] == pair), 0)
    combined_score = (jaccard_score + adamic_adar_score + resource_allocation_score) / 3
    combined_scores[pair] = combined_score

# Sort the combined scores and calculate precision
combined_scores_sorted = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
combined_precision = calculate_precision([(x[0][0], x[0][1], x[1]) for x in combined_scores_sorted], test_edges_set)

combined_precision

0.9333333333333333

### Part 7: Applying Machine Learning to Link Prediction

#### Objective
Building on similarity scores and structural features of the network, we can apply machine learning models to further enhance link prediction accuracy. This approach allows us to systematically combine various features and learn from the data to predict the existence of links between nodes.

#### Implementation
We use a RandomForestClassifier, a robust and widely used machine learning model, to predict links. The model is trained on features derived from our similarity metrics, specifically the combined score which encapsulates multiple aspects of node similarity.

1. **Feature Preparation**: We create a dataset where each instance represents a pair of nodes. The features include our combined similarity score, and the target variable indicates whether a link exists between the nodes.

2. **Model Training**: We split our dataset into training and testing sets to evaluate the model's performance accurately.

3. **Prediction and Evaluation**: The trained model is used to predict links on the test set. We assess the model's accuracy, providing insight into its predictive capability.

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Generate positive (existing links) and negative (non-existing links) samples for training
positive_samples = [(source, target, combined_scores.get((source, target), 0), 1) for source, target in test_edges.to_numpy()]
negative_samples = [(pair[0], pair[1], combined_scores.get(pair, 0), 0) for pair in node_pairs if pair not in test_edges_set][:len(positive_samples)]
ml_samples = positive_samples + negative_samples

# Convert to DataFrame and prepare the dataset
ml_df = pd.DataFrame(ml_samples, columns=["source", "target", "combined_score", "link_exists"])
X = ml_df[["combined_score"]]
y = ml_df["link_exists"]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Train a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set and evaluate accuracy
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

accuracy

0.9761167624944714